In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import requests
from tqdm import tqdm
import time
import os
import re
import datetime

In [3]:
def getTables(response):
    # HTTPステータスコードが200（成功）の場合のみ処理を続行
    if response.getcode() == 200:
        # HTMLをパースしてBeautifulSoupオブジェクトを作成
        bs = BeautifulSoup(response, 'html.parser')
        bs = bs.decode('UTF-8')
        html_string = str(bs)
        html_io = StringIO(html_string)

        # テーブルデータを抽出
        tables = pd.read_html(html_io)
        df = tables[0]
        print(df)
        return df

    else:
        print(f"HTTPステータスコード {response.getcode()}: ページの取得に失敗しました")
        

In [4]:
#開催年
years = [str(i).zfill(4) for i in range(2021, 2022)]
#開催場所 01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09::阪神, 10:小倉
places = [str(i).zfill(2) for i in range (5, 6)]
#開催回
times = [str(i).zfill(2) for i in range(1, 2)]
#開催日
days = [str(i).zfill(2) for i in range(1, 2)]
#レースNo
races = [str(i).zfill(2) for i in range(1, 13)]

raceIdList = []
for y in years:
    for p in places:
        for t in times:
            for d in days:
                for r in races:
                    raceIdList.append(y + p + t + d + r)
                    

In [5]:

url = 'https://db.netkeiba.com/race/'
colName = ['raceId', 'htmlBytes']
df = pd.DataFrame(columns=colName)
escapeList = []

# pickleファイルが存在するか確認し、データを読み込む
if os.path.isfile('race_html.pkl'):
    df = pd.read_pickle('race_html.pkl')
    escapeList = df['raceId'].to_list()

# ページネーション用の競走IDリストを生成する関数
def addEscapeList(id: str, ll: list):
    idAry = [id[0:4], id[4:6], id[6:8], id[8:10], id[10:12]]
    for r in range(1, 13):
        idAry[4] = str(r).zfill(2)
        ll.append(''.join(idAry))
    if idAry[3] == '01':
        for d in range(2, 9):
            idAry[3] = str(d).zfill(2)
            ll = addEscapeList(''.join(idAry), ll)
    if idAry[2] == '01':
        for t in range(2, 9):
            idAry[2] = str(t).zfill(2)
            ll = addEscapeList(''.join(idAry), ll)

    return ll

# raceIdListに実際の競走IDのリストがあると仮定します
# raceIdList = ...

for raceId in tqdm(raceIdList):
    try:
        if raceId in escapeList:
            continue
        response = url + raceId
        html = requests.get(response)
        #html.encoding = 'utf-8'
        soup = BeautifulSoup(html.content, 'html.parser')

        if 'レース結果' in soup.text:
            tmpDf = pd.DataFrame([[raceId, html.content]], columns=colName)  # 'colmuns'の誤りを修正
            df = pd.concat([df, tmpDf], axis=0, ignore_index=True)
        else:
            escapeList = addEscapeList(raceId, escapeList)  # 変数名を修正

        time.sleep(3)
    except Exception as e:  # 例外をキャッチしてログに記録
        print(f'例外が発生しました: {str(e)}')

# DataFrameをpickleファイルに保存
df.to_pickle('race_html.pkl')


100%|██████████| 12/12 [00:00<?, ?it/s]


In [6]:
htmlDf = pd.read_pickle('race_html.pkl')

raceInfoList = []

#raceResultDf = pd.DataFrame()
for idx, dat in tqdm(htmlDf.iterrows(), total=len(htmlDf)):
    raceId = dat['raceId']
    htmlBytes = dat['htmlBytes']

    soup = BeautifulSoup(htmlBytes.decode('euc-jp','ignore'), 'html.parser')
    mainrace_data = soup.find('div', class_='mainrace_data')
    rowdata = {}
    rowdata['raceId'] = raceId
    rowdata['レース名'] = mainrace_data.find('h1').text
    rowdata['R'] = mainrace_data.find('dt').text.replace('\n','').replace(' ','').replace('R', '')
    spantexts = mainrace_data.find('span').text.replace('\xa0','').replace(' ','').split('/')
    
    rowdata['コース種'] = '障害' if '障' in spantexts[0] else 'ダート' if 'ダ' in spantexts[0] else '芝'
    
    rowdata['コース回り'] = '右' if '右' in spantexts[0] else '左' if '左' in spantexts[0] else '障害'

    rowdata['距離'] = int(re.findall('\d+',spantexts[0])[0])

    rowdata['天気'] = spantexts[1][3:]

    rowdata['馬場'] = spantexts[2].split(':')[1]
    rowdata['発走'] = spantexts[3][3:]

    smalltxt = mainrace_data.find('p', class_='smalltxt').text.replace('\xa0',' ').replace('  ',' ').split(' ')
    dt = datetime.datetime.strptime(smalltxt[0], '%Y年%m月%d日')
    rowdata['日付'] = dt.strftime('%Y/%m/%d')

    placeDict = {
        '01':'札幌', '02':'函館', '03':'福島', '04':'新潟', '05':'東京', '06':'中山', '07':'中京', '08':'京都', '09':'阪神', '10':'小倉'
    }
    rowdata['開催場所'] = placeDict[raceId[4:6]]

    if 'G1' in rowdata['レース名']:
        raceGrade = 'G1'
    elif 'G2' in rowdata['レース名']:
        raceGrade = 'G2'
    elif 'G3' in rowdata['レース名']:
        raceGrade = 'G3'
    elif '未勝利' in smalltxt[2]:
        raceGrade = '未勝利'
    elif '新馬' in smalltxt[2]:
        raceGrade = '新馬'
    elif '1勝クラス' in smalltxt[2] or '500万' in smalltxt[2]:
        raceGrade = '1勝クラス'
    elif '2勝' in smalltxt[2] or '1000万' in smalltxt[2]:
        raceGrade = '2勝クラス'
    elif '3勝' in smalltxt[2] or '1600万' in smalltxt[2]:   
        raceGrade = '3勝クラス'
    else:
        raceGrade = 'オープン'  

    if '牡・牝' in smalltxt[3]:
        restriction = '牡・牝'
    elif '牝' in smalltxt[3]:
        restriction = '牝'
    elif '牡' in smalltxt[3]:
        restriction = '牡'
    else:
        restriction = '無'

    rowdata['制限'] = restriction

    if 'ハンデ' in smalltxt[3]:
        handicap = 'ハンデ'
    elif '別定' in smalltxt[3]:
        handicap = '別定'
    else:
        handicap = '定量'
    
    rowdata['ハンデ'] = handicap

    raceInfoList.append(rowdata)
raceInfoDf = pd.DataFrame(raceInfoList)
raceInfoDf.to_pickle('race_info.pkl')

100%|██████████| 12/12 [00:00<00:00, 36.77it/s]


In [7]:
raceResultDf = pd.read_pickle('race_info.pkl')
raceResultDf

,raceId,レース名,R,コース種,コース回り,距離,天気,馬場,発走,日付,開催場所,制限,ハンデ
0,202105010101,3歳未勝利,1,ダート,左,1400,晴,重,10:10,2021/01/30,東京,牝,定量
1,202105010102,3歳新馬,2,ダート,左,1400,晴,重,10:40,2021/01/30,東京,無,定量
2,202105010103,3歳未勝利,3,ダート,左,1600,晴,重,11:10,2021/01/30,東京,無,定量
3,202105010104,3歳1勝クラス,4,ダート,左,1600,晴,重,11:40,2021/01/30,東京,無,定量
4,202105010105,3歳未勝利,5,芝,左,1800,晴,良,12:30,2021/01/30,東京,無,定量
5,202105010106,3歳新馬,6,芝,左,1600,晴,良,13:00,2021/01/30,東京,無,定量
6,202105010107,4歳以上1勝クラス,7,ダート,左,1400,晴,重,13:30,2021/01/30,東京,無,定量
7,202105010108,4歳以上2勝クラス,8,ダート,左,2100,晴,重,14:01,2021/01/30,東京,無,定量
8,202105010109,銀蹄ステークス,9,ダート,左,1400,晴,重,14:35,2021/01/30,東京,無,ハンデ
9,202105010110,クロッカスステークス(L),10,芝,左,1400,晴,良,15:10,2021/01/30,東京,無,別定


In [16]:
table = soup.find_all('table')[0]

columns = []

for head in table.find_all('th'):
    columns.append(head.text)
    print(head.text)

columns = ['raceId'] + columns + ['horseId', 'jockeyId', 'trainerId']
df = pd.DataFrame(columns=columns)

for i, row in enumerate(table.find_all('tr')):
    if i == 0:
        continue
    items = [raceId]

    cells = row.find_all('td')

    for cell in cells:
        items.append(cell.text.replace('\n',''))

    items.append(str(cells[3]).split('/horse/')[1].split('/')[0])
    items.append(str(cells[6]).split('/recent/')[1].split('/')[0])
    items.append(str(cells[18]).split('/recent')[1].split('/')[0])

    df.loc[i] = items
    print(items)

raceResultDf = pd.concat([raceResultDf, df],axis=0)


着順
枠番
馬番
馬名
性齢
斤量
騎手
タイム
着差
ﾀｲﾑ指数
通過
上り
単勝
人気
馬体重
調教ﾀｲﾑ
厩舎ｺﾒﾝﾄ
備考
調教師
馬主
賞金(万円)
['202105010112', '1', '6', '11', 'ヤシャマル', '牡4', '56', '横山武史', '1:48.5', '', '**', '6-4-3', '33.4', '2.2', '1', '510(+14)', '', '', '', '[東]尾形和幸', '高橋文男', '760.0', '2017101201', '01170', '']
['202105010112', '2', '7', '13', 'ダイワクンナナ', '牝4', '54', '田辺裕信', '1:48.7', '1.1/4', '**', '10-9-8', '33.2', '11.6', '5', '496(0)', '', '', '', '[東]国枝栄', '大城正一', '300.0', '2017104809', '01075', '']
['202105010112', '3', '4', '7', 'フィリアーノ', '牡5', '57', '津村明秀', '1:49.0', '1.3/4', '**', '2-3-3', '33.9', '29.3', '8', '476(+4)', '', '', '', '[東]手塚貴久', '吉田勝己', '190.0', '2016104818', '01092', '']
['202105010112', '4', '7', '12', 'ウインアルカンナ', '牝4', '54', '石川裕紀', '1:49.0', 'アタマ', '**', '1-1-1', '34.2', '34.2', '9', '460(+2)', '', '', '', '[東]武市康男', 'ウイン', '110.0', '2017101417', '01150', '']
['202105010112', '5', '8', '14', 'ビバヴィットーリオ', '牡4', '56', '川田将雅', '1:49.1', '1/2', '**', '2-2-2', '34.2', '10.2', '4', '530(+6)', '', '', '', '[

In [11]:
raceResultDf


,raceId,レース名,R,コース種,コース回り,距離,天気,馬場,発走,日付,...,馬体重,調教ﾀｲﾑ,厩舎ｺﾒﾝﾄ,備考,調教師,馬主,賞金(万円),horseId,jockeyId,trainerId
0,202105010101,3歳未勝利,1,ダート,左,1400.0,晴,重,10:10,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202105010102,3歳新馬,2,ダート,左,1400.0,晴,重,10:40,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202105010103,3歳未勝利,3,ダート,左,1600.0,晴,重,11:10,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202105010104,3歳1勝クラス,4,ダート,左,1600.0,晴,重,11:40,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202105010105,3歳未勝利,5,芝,左,1800.0,晴,良,12:30,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,202105010106,3歳新馬,6,芝,左,1600.0,晴,良,13:00,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,202105010107,4歳以上1勝クラス,7,ダート,左,1400.0,晴,重,13:30,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,202105010108,4歳以上2勝クラス,8,ダート,左,2100.0,晴,重,14:01,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,202105010109,銀蹄ステークス,9,ダート,左,1400.0,晴,重,14:35,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,202105010110,クロッカスステークス(L),10,芝,左,1400.0,晴,良,15:10,2021/01/30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.to_csv('data.csv',encoding='utf-8')
df.columns
print(df)

         raceId 着順 枠番  馬番     馬名  性齢  斤量    騎手     タイム 着差  ...       馬体重  \
1  202105010112  1  6  11  ヤシャマル  牡4  56  横山武史  1:48.5     ...  510(+14)   

  調教ﾀｲﾑ 厩舎ｺﾒﾝﾄ 備考      調教師    馬主 賞金(万円)     horseId jockeyId trainerId  
1                  [東]尾形和幸  高橋文男  760.0  2017101201    01170            

[1 rows x 25 columns]
